# SedonaDB vs. GeoPandas

This notebook demonstrates a SedonaDB query vs. an equivalent query in GeoPandas.

It uses the data from [SpatialBench](https://sedona.apache.org/spatialbench/).

## SedonaDB query

In [1]:
import sedona.db

In [2]:
sd = sedona.db.connect()

In [3]:
data_dir = "/Users/matthewpowers/data"

In [4]:
sd.read_parquet(f"{data_dir}/sf1-parquet/building.parquet").to_view("building")
sd.read_parquet(f"{data_dir}/sf1-parquet/trip.parquet").to_view("trip")

In [5]:
sql = """
SELECT b.b_buildingkey, b.b_name, COUNT(*) AS nearby_pickup_count
FROM trip t 
JOIN building b ON ST_DWithin(ST_GeomFromWKB(t.t_pickuploc), ST_GeomFromWKB(b.b_boundary), 0.0045) -- ~500m
GROUP BY b.b_buildingkey, b.b_name
ORDER BY nearby_pickup_count DESC, b.b_buildingkey ASC
"""

In [6]:
%%time
sd.sql(sql).show(5)

┌───────────────┬───────────┬─────────────────────┐
│ b_buildingkey ┆   b_name  ┆ nearby_pickup_count │
│     int64     ┆    utf8   ┆        int64        │
╞═══════════════╪═══════════╪═════════════════════╡
│          3779 ┆ linen     ┆                  42 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│         19135 ┆ misty     ┆                  36 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│          4416 ┆ sienna    ┆                  26 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│         12627 ┆ salmon    ┆                  22 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│          1906 ┆ burlywood ┆                  19 │
└───────────────┴───────────┴─────────────────────┘
CPU times: user 1.07 s, sys: 38 ms, total: 1.11 s
Wall time: 336 ms


In [7]:
sd.sql(sql).count()

375

## GeoPandas query

In [8]:
import pandas as pd
import geopandas as gpd

In [9]:
trips_df = pd.read_parquet(f"{data_dir}/sf1-parquet/trip.parquet")
trips_df["pickup_geom"] = gpd.GeoSeries.from_wkb(
    trips_df["t_pickuploc"], crs="EPSG:4326"
)
pickups_gdf = gpd.GeoDataFrame(trips_df, geometry="pickup_geom", crs="EPSG:4326")

In [10]:
buildings_df = pd.read_parquet(f"{data_dir}/sf1-parquet/building.parquet")
buildings_df["boundary_geom"] = gpd.GeoSeries.from_wkb(
    buildings_df["b_boundary"], crs="EPSG:4326"
)
buildings_gdf = gpd.GeoDataFrame(
    buildings_df, geometry="boundary_geom", crs="EPSG:4326"
)

In [11]:
threshold = 0.0045  # degrees (~500m)
result = (
    buildings_gdf.sjoin(pickups_gdf, predicate="dwithin", distance=threshold)
    .groupby(["b_buildingkey", "b_name"], as_index=False)
    .size()
    .rename(columns={"size": "nearby_pickup_count"})
    .sort_values(["nearby_pickup_count", "b_buildingkey"], ascending=[False, True])
    .reset_index(drop=True)
)
print(result)

     b_buildingkey     b_name  nearby_pickup_count
0             3779      linen                   42
1            19135      misty                   36
2             4416     sienna                   26
3            12627     salmon                   22
4             1906  burlywood                   19
..             ...        ...                  ...
370          19888      black                    1
371          19941       blue                    1
372          19951      lemon                    1
373          19959    magenta                    1
374          19996      black                    1

[375 rows x 3 columns]
